In [2]:
!ls

colleges.db                  social.db
movies_and_ratings.db        stanford_sql_exercises.Rmd
readme.md                    stanford_sql_exercises.ipynb


In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('movies_and_ratings.db')

## First Exercise

In [4]:
# Find the titles of all movies directed by Steven Spielberg. 
pd.read_sql_query('''
select title 
from Movie 
where director = "Steven Spielberg"
''', conn)

,title
0,E.T.
1,Raiders of the Lost Ark


In [5]:
# Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order. 
pd.read_sql_query('''
select distinct year
from Movie, Rating 
where Movie.mID = Rating.mID and stars >= 4
order by year 
''', conn)

,year
0,1937
1,1939
2,1981
3,2009


In [6]:
# Find the titles of all movies that have no ratings. 
pd.read_sql_query('''
select distinct title
from Movie 
join Rating
where Movie.mID not in (select mID from Rating) 
''', conn)

,title
0,Star Wars
1,Titanic


In [7]:
# Some reviewers didn't provide a date with their rating. Find the names of all reviewers who have ratings with a NULL
# value for the date. 
pd.read_sql_query('''
select name from Reviewer, Rating 
where Reviewer.rID = Rating.rID and ratingDate is null
''', conn)


,name
0,Daniel Lewis
1,Chris Jackson


In [8]:
# Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. 
# Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars. 
pd.read_sql_query('''
select name, title, stars, ratingDate
from Reviewer, (select * from Rating, Movie where Rating.mID = Movie.mID) as T where T.rID = Reviewer.rID
order by name, title, stars
''',conn)


,name,title,stars,ratingDate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


In [9]:
# For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return 
# the reviewer's name and the title of the movie. 

pd.read_sql_query('''
select name, title
from Movie,
(select *, (select count(*) from Rating where Rating.rID = R.rID and Rating.mID = R.mID 
    and Rating.ratingDate > R.ratingDate and Rating.stars > R.stars) as counting
    from Rating as R
    where counting >= 1
) as R2, 
Reviewer
where Movie.mID = R2.mID and Reviewer.rID = R2.rID
''', conn)


,name,title
0,Sarah Martinez,Gone with the Wind


In [10]:
# For each movie that has at least one rating, find the highest number of stars that movie received. Return the movie title
# and number of stars. Sort by movie title. 

pd.read_sql_query('''
select Title, max(stars) as super_stars 
from Rating,Movie 
where Rating.mID = Movie.mID 
group by Rating.mID 
order by Title
''', conn)


,title,super_stars
0,Avatar,5
1,E.T.,3
2,Gone with the Wind,4
3,Raiders of the Lost Ark,4
4,Snow White,5
5,The Sound of Music,3


In [11]:
# For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings 
# given to that movie. Sort by rating spread from highest to lowest, then by movie title.
pd.read_sql_query('''
select Title, max(stars) - min(stars) as spread
from Rating, Movie
where Rating.mID = Movie.mID
group by Rating.mID
order by spread desc, Title
''', conn)

,title,spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


In [12]:
# Find the difference between the average rating of movies released before 1980 and the average rating of movies 
# released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for 
# movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.) 

pd.read_sql_query('''
select a2- a1
from
    (select avg(avg_year) as a1 from
        (select year, 
        avg(avg_stars) as avg_year 
        from (select Rating.mID, avg(stars) as avg_stars, year from Rating, Movie where Rating.mID = Movie.mID group by Rating.mID) 
        group by year) as before_1980
    where year > 1980) as b1980,

    (select avg(avg_year) as a2 from
        (select year, 
        avg(avg_stars) as avg_year 
        from (select Rating.mID, avg(stars) as avg_stars, year from Rating, Movie where Rating.mID = Movie.mID group by Rating.mID) 
        group by year) as after_1980
    where year < 1980) as a1980
''', conn)


,a2- a1
0,0.055556


## Second Exercise

In [13]:
pd.read_sql_query('''
select * 
from sqlite_master
where type = 'table'
''', conn)

,type,name,tbl_name,rootpage,sql
0,table,Movie,Movie,2,"CREATE TABLE Movie(mID int, title text, year i..."
1,table,Reviewer,Reviewer,3,"CREATE TABLE Reviewer(rID int, name text)"
2,table,Rating,Rating,4,"CREATE TABLE Rating(rID int, mID int, stars in..."


In [14]:
# Find the names of all reviewers who rated Gone with the Wind. 
pd.read_sql_query('''
select distinct name
from Movie, (select * from Rating, Reviewer where Rating.rID = Reviewer.rID) as T1
where Movie.mID = T1.mID and Movie.mID = 101
''', conn)

,name
0,Sarah Martinez
1,Mike Anderson


In [15]:
# For any rating where the reviewer is the same as the director of the movie, return the reviewer name, movie title, and number of stars. 
pd.read_sql_query('''
select director, title, stars 
from Movie, (select * from Rating, Reviewer where Rating.rID = Reviewer.rID) as T1 
where Movie.mID = T1.mID and name = director
''', conn)

,director,title,stars
0,James Cameron,Avatar,5


In [16]:
# Return all reviewer names and movie names together in a single list, alphabetized. (Sorting by the first name of the 
# reviewer and first word in the title is fine; no need for special processing on last names or removing "The".) 

pd.read_sql_query('''
select  distinct name
from Reviewer 
union 
select distinct title
from Movie
''', conn)


,name
0,Ashley White
1,Avatar
2,Brittany Harris
3,Chris Jackson
4,Daniel Lewis
5,E.T.
6,Elizabeth Thomas
7,Gone with the Wind
8,James Cameron
9,Mike Anderson


In [17]:
# Find the titles of all movies not reviewed by Chris Jackson. 
pd.read_sql_query('''
select title
from Movie
except 
select title 
from Rating, Movie 
where  Rating.mID = Movie.mID and rID = 205
''', conn)

,title
0,Avatar
1,Gone with the Wind
2,Snow White
3,Star Wars
4,Titanic


In [18]:
# For all pairs of reviewers such that both reviewers gave a rating to the same movie, return the names of both reviewers. 
# Eliminate duplicates, don't pair reviewers with themselves, and include each pair only once.
# For each pair, return the names in the pair in alphabetical order. 

pd.read_sql_query('''
select distinct T1.name, T2.name from
(select name, mID from Rating, Reviewer where Rating.rID = Reviewer.rID order by name) as T1,
(select name, mID from Rating, Reviewer where Rating.rID = Reviewer.rID order by name) as T2
where T1.mID = T2.mID and T1.name < T2.name
''', conn)

,name,name
0,Ashley White,Chris Jackson
1,Brittany Harris,Chris Jackson
2,Daniel Lewis,Elizabeth Thomas
3,Elizabeth Thomas,James Cameron
4,Mike Anderson,Sarah Martinez


In [19]:
# List movie titles and average ratings, from highest-rated to lowest-rated. If two or more movies have the same 
# average rating, list them in alphabetical order. 

pd.read_sql_query('''
select title, avg(stars) from Movie,
Rating where Movie.mID = Rating.mID
group by Rating.mID
order by avg(stars) desc, title
''', conn)


,title,avg(stars)
0,Snow White,4.500000
1,Avatar,4.000000
2,Raiders of the Lost Ark,3.333333
3,Gone with the Wind,3.000000
4,E.T.,2.500000
5,The Sound of Music,2.500000


In [20]:
# For each rating that is the lowest (fewest stars) currently in the database, return the reviewer name, movie title, and 
# number of stars. 
pd.read_sql_query('''
select name, title, stars
from Movie,
(select mID, stars, name from Rating, Reviewer where Reviewer.rID = Rating.rID and stars = (select min(stars) from Rating)) as T
where Movie.mID = T.mID
''', conn)


,name,title,stars
0,Sarah Martinez,Gone with the Wind,2
1,Brittany Harris,The Sound of Music,2
2,Brittany Harris,Raiders of the Lost Ark,2
3,Chris Jackson,E.T.,2


In [21]:
# Find the names of all reviewers who have contributed three or more ratings.
pd.read_sql_query('''
select name
from Rating, Reviewer
where Rating.rID = Reviewer.rID
group by Rating.rID
having count(*) >= 3
''', conn)

,name
0,Brittany Harris
1,Chris Jackson


In [22]:
# Some directors directed more than one movie. For all such directors, return the titles of all movies directed by them, along with the director name.
# Sort by director name, then movie title.
pd.read_sql_query('''
select title, Movie.director from Movie,
 (select director from Movie group by director having count(*) >= 2) as T
 where Movie.director = T.director
''', conn)

,title,director
0,E.T.,Steven Spielberg
1,Titanic,James Cameron
2,Avatar,James Cameron
3,Raiders of the Lost Ark,Steven Spielberg


In [23]:
# Find the movie(s) with the highest average rating. Return the movie title(s) and average rating. 
pd.read_sql_query('''
select title, avg(stars) from
Movie,
Rating
where Movie.mID = Rating.mID
group by Rating.mID
having avg(stars) = (select max(avg_stars) as avg_stars from 
(select mID, avg(stars) as avg_stars from Rating group by mID))
''', conn)

,title,avg(stars)
0,Snow White,4.5


In [24]:
# Find the movie(s) with the lowest average rating. Return the movie title(s) and average rating.
pd.read_sql_query('''
select title, avg(stars) from
Movie,
Rating
where Movie.mID = Rating.mID
group by Rating.mID
having avg(stars) = (select min(avg_stars) as avg_stars from 
(select mID, avg(stars) as avg_stars from Rating group by mID))
''', conn)

,title,avg(stars)
0,The Sound of Music,2.5
1,E.T.,2.5


In [25]:
# For each director, return the director's name together with the title(s) of the movie(s) they directed that received 
# the highest rating among all of their movies, and the value of that rating.
# Ignore movies whose director is NULL. 

pd.read_sql_query('''
select director, title, max(stars) from Movie, Rating  
where Movie.mID = Rating.mID and director is not null
group by director
''', conn)


,director,title,max(stars)
0,James Cameron,Avatar,5
1,Robert Wise,The Sound of Music,3
2,Steven Spielberg,Raiders of the Lost Ark,4
3,Victor Fleming,Gone with the Wind,4


## SQL Social-Network Query Exercises

In [26]:
# read the new database for this exercise
conn2 = sqlite3.connect('social.db')
pd.read_sql_query('''
select * from sqlite_master where type='table' 
''', conn2)

,type,name,tbl_name,rootpage,sql
0,table,Highschooler,Highschooler,2,"CREATE TABLE Highschooler(ID int, name text, g..."
1,table,Friend,Friend,3,"CREATE TABLE Friend(ID1 int, ID2 int)"
2,table,Likes,Likes,4,"CREATE TABLE Likes(ID1 int, ID2 int)"


In [27]:
# Find the names of all students who are friends with someone named Gabriel. 
pd.read_sql_query('''
select name
from Friend, Highschooler
where id1  in (select id from Highschooler where name = "Gabriel") and Friend.ID2 = Highschooler. ID
''', conn2)

,name
0,Cassandra
1,Andrew
2,Jessica
3,Jordan
4,Alexis


In [28]:
# For every student who likes someone 2 or more grades younger than themselves, return that student's name and 
# grade, and the name and grade of the student they like. 

pd.read_sql_query('''
select H.name, H.grade, H2.name, H2.grade from Highschooler as H
inner join Likes on H.ID = Likes.ID1
inner join Highschooler as H2 on ID2 = H2.ID
where ID2 in (select ID from Highschooler where grade = H.grade - 2)
''', conn2)

,name,grade,name,grade
0,John,12,Haley,10


In [29]:
# For every pair of students who both like each other, return the name and grade of both students. Include each 
# pair only once, with the two names in alphabetical order. 

pd.read_sql_query(''' 
select *
from
(select H1.name as n1 , H1.grade as g1, H2.name as n2, H2.grade as g2 from Likes
inner join Highschooler as H1 on H1.id = Likes.id1
inner join Highschooler as H2 on H2.id = Likes.id2

intersect

select H2.name, H2.grade,H1.name, H1.grade from Likes
inner join Highschooler as H1 on H1.id = Likes.id1
inner join Highschooler as H2 on H2.id = Likes.id2) as T
where n1 < n2
''', conn2)

,n1,g1,n2,g2
0,Cassandra,9,Gabriel,9
1,Jessica,11,Kyle,12


In [30]:
# Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their 
# names and grades. Sort by grade, then by name within each grade. 

pd.read_sql_query('''
select name, grade from Highschooler
where id in
(select id from Highschooler 
except 
select id1 from Likes
except
select id2 from Likes)
''', conn2)


,name,grade
0,Jordan,9
1,Tiffany,9
2,Logan,12


In [31]:
# For every situation where student A likes student B, but we have no information about whom B likes 
# (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades.

pd.read_sql_query('''
select h1.name, h1.grade, h2.name, h2.grade from Likes as L
inner join Highschooler h1 on h1.id = L.id1
inner join Highschooler h2 on h2.id = L.id2
where L.id2 not in (select id1 from Likes)
order by h1.name
''', conn2)

,name,grade,name,grade
0,Alexis,11,Kris,10
1,Austin,11,Jordan,12
2,Brittany,10,Kris,10
3,John,12,Haley,10


In [32]:
# Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, 
# then by name within each grade. 

pd.read_sql_query('''
select distinct H1.name, H1.grade
from Friend as F
inner join Highschooler as H1 on H1.id = F.id1
inner join Highschooler as H2 on H2.id = F.id2
where (
select id1 from Friend
inner join Highschooler as HH2 on HH2.id = Friend.id2
where id1 = F.id1 and HH2.grade = H1.grade
) not null
and
(
select id1 from Friend
inner join Highschooler as HH2 on HH2.id = Friend.id2
where id1 = F.id1 and HH2.grade <> H1.grade
) is null
order by h1.grade, h1.name
''', conn2)


,name,grade
0,Jordan,9
1,Brittany,10
2,Haley,10
3,Kris,10
4,Gabriel,11
5,John,12
6,Logan,12


In [33]:
# For each student A who likes a student B where the two are not friends, find if they have a friend C in common 
# (who can introduce them!). For all such trios, return the name and grade of A, B, and C.

pd.read_sql_query('''
select name1, g1, name2, g2, name, grade
from Friend as Fi,
(select id1 as fid1, h1.name as name1, h1.grade as g1, id2 as fid2, h2.name as name2, h2.grade as g2
from Likes as L
inner join Highschooler as H1 on H1.id = L.id1
inner join Highschooler as H2 on H2.id = L.id2
where (select id1
        from Friend
        where Friend.ID1 = L.ID1 and Friend.ID2 = L.ID2) is null
and
        (select id2 from Friend where id1 = L.id1
        intersect
        select id2 from Friend where id1 = L.id2
        ) not null)
inner join Highschooler on Highschooler.id = Fi.id2
where ID1 = fid1 and ID in (select id2 from Friend where ID1 = fid2)''', conn2)

,name1,g1,name2,g2,name,grade
0,Andrew,10,Cassandra,9,Gabriel,9
1,Austin,11,Jordan,12,Andrew,10
2,Austin,11,Jordan,12,Kyle,12


In [34]:
# Find the difference between the number of students in the school and the number of different first names. 
pd.read_sql_query(''' 
select count(id) - count(distinct name) from Highschooler
''', conn2)

,count(id) - count(distinct name)
0,2


In [35]:
# Find the name and grade of all students who are liked by more than one other student. 
pd.read_sql_query('''
select name, grade
from Likes
inner join Highschooler as H on H.id = Likes.id2
group by ID2 having count(id1) >=2
''', conn2)

,name,grade
0,Kris,10
1,Cassandra,9


## Extra queries from social network

In [44]:
# For every situation where student A likes student B, but student B likes a different student C, 
# return the names and grades of A, B, and C. 

pd.read_sql_query('''
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade
from Likes as L 
inner join Likes as L2 on L.id2 = L2.id1
inner join Highschooler as H1 on H1.id = L.id1
inner join Highschooler as H2 on H2.id = L.id2
inner join Highschooler as H3 on H3.id = L2.id2
where L2.id2 <> L.id1
''', conn2)


,name,grade,name,grade,name,grade
0,Andrew,10,Cassandra,9,Gabriel,9
1,Gabriel,11,Alexis,11,Kris,10


In [54]:
# Find those students for whom all of their friends are in 
# different grades from themselves. Return the students' names and 
# grades.

pd.read_sql_query('''
select distinct name, grade
from Friend
inner join Highschooler on Highschooler.id = Friend.id1
where (select id2
    from friend as F
    inner join Highschooler as H on H.id = F.id2
    where F.Id1 = Friend.id1 and Highschooler.grade <> H.grade
) not null
and 
(select id2
    from friend as F
    inner join Highschooler as H on H.id = F.id2
    where F.Id1 = Friend.id1 and Highschooler.grade = H.grade
) is null
''', conn2)


,name,grade
0,Austin,11


In [61]:
# What is the average number of friends per student? (Your result 
# should be just one number.) 

pd.read_sql_query('''
select avg(friend_count)
from (
select *, count(id2) as friend_count
from Friend
inner join highschooler on friend.id1 = highschooler.id
group by friend.id1
)
''', conn2)

,avg(friend_count)
0,2.5


In [70]:
# Find the number of students who are either friends with Cassandra
# or are friends of friends of Cassandra. Do not count Cassandra, 
# even though technically she is a friend of a friend.

pd.read_sql_query('''
select count(*) from (
select id2 from
Friend
inner join (select id2 as cas_friends from Friend where id1 = 1709) as cas
on id1 = cas.cas_friends
where id2 <> 1709

union

select id2 as cas_friends from Friend where id1 = 1709
)


''', conn2)

,count(*)
0,7


In [89]:
# Find the name and grade of the student(s) with the greatest 
# number of friends. 

pd.read_sql_query('''
select name, grade
from Friend
inner join Highschooler as H on H.id = Friend.id1
group by id1
having count(id2) = (select max(number)from

(select count(id2) as number
from Friend
group by id1))

''', conn2)

,name,grade
0,Alexis,11
1,Andrew,10
